In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from network.models.baseline import BaseShip
X = torch.load('/home/rmadeye/kaggle/spaceship/data/inputs/X.pt')
y = torch.load('/home/rmadeye/kaggle/spaceship/data/inputs/y.pt')

In [2]:
# torch.tensor(X)

In [3]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
epochs = 20
batch_size = 64
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
model = BaseShip(input_dim=13, hidden_dim=64).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
train_loader = DataLoader([(x, y) for x, y in zip(X_train, y_train)], batch_size=batch_size, shuffle=True)
test_loader = DataLoader([(x, y) for x, y in zip(X_test, y_test)], batch_size=batch_size, shuffle=True)

In [5]:
# Number of epochs
n_epochs = 10

# Start training
for epoch in range(n_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.float().to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.float().to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    print(f'Epoch: {epoch+1}/{n_epochs}, Loss: {train_loss:.4f}, Accuracy: {accuracy*100:.2f}%')


IndexError: too many indices for tensor of dimension 2